<a href="https://colab.research.google.com/github/Bambika69/KATICA/blob/main/Prep%26FineTune/KATICAv0dot5_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.databricks.com/blog/2023/03/24/hello-dolly-democratizing-magic-chatgpt-open-models.html

In [1]:
!pip -q install datasets loralib sentencepiece
!pip -q install git+https://github.com/huggingface/transformers
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 10

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [3]:
from peft import PeftModel
from transformers import AutoTokenizer, GPTJForCausalLM, GenerationConfig

model = GPTJForCausalLM.from_pretrained(
    "EleutherAI/gpt-j-6B",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "Bandika/KATICAv0dot5")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [4]:
def KATICA_generate(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )

    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=128,
        pad_token_id = 0,
        eos_token_id = 50256
    )

    for s in generation_output.sequences:
        print(tokenizer.decode(s))

In [5]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Milyen színű az alma?
### Response:"""

KATICA_generate(PROMPT)

Generating...
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Milyen színű az alma?
### Response:
A mély, vízi és nyálkás állatok közületei egymaga. A fajtárakban lakóhelyeikben élnek, amit csodálatosan megfigyelhetjünk. Az almadarabot tudunk használni arra, hogy elszigetelt legyenek vagy biztonságban maradjon. Ez lehetővé teszik számukra, h


In [6]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Hány bábu van egy sakktáblán?
### Response:"""

KATICA_generate(PROMPT)

Generating...
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Hány bábu van egy sakktáblán?
### Response:
Ez azt jelenti, hogy néhány kötelező érvvel kell rendezni a törlésre vonatkozó dolgot. A legjobb esetben úgy mondhatjuk "nincs" vagy "egy". Ha valaki megkérdezi, hogy mit gondolnak arról, hogy "a", akkor nem lehetne őket visszaemlékezni arra, hogy mi volt a helyes vá


In [7]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Egyszer volt hol nem volt
### Response:"""

KATICA_generate(PROMPT)

Generating...
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Egyszer volt hol nem volt
### Response:
Nem volt egyszer, de azt mondta nekem hogy "nincs" és ezért elmentettem. Ezután következőket tettek hozzám: "Sokat szerettem volna lenni". Aztán megismételtük: "Kell valami jól vagy rosszul?" Értem önt!
### Training Input:
A nagyobbik csoportban van olyan emberek, akiknek mind


In [8]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Írj egy emailt a főnökömnek, hogy megbetegedtem
### Response:"""

KATICA_generate(PROMPT)

Generating...
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Írj egy emailt a főnökömnek, hogy megbetegedtem
### Response:
Ön számíthat arra, hogy az üzenet küldése elhalasztásra kerülheti. A feladatok visszaigazolása és a fejlődés érdekében minden lehetséges eszközzel kell törvénnyel rendezni. Az összes probléma megoldásának nyomon követési módszerét használva bizonyosan javul


In [9]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Write me a C++ code that generates 5 random numbers
### Response:"""

KATICA_generate(PROMPT)

Generating...
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Write me a C++ code that generates 5 random numbers
### Response:
#include <iostream>
using namespace std;
int main() {
  int num1,num2,num3,num4,num5;

  // Generate five random integers between 0 and 100 inclusive
  do{
    cin>>num1>>num2>>num3>>num4>>num5;
  }while(cin);

  return 0;
}
### Explanation:
The program reads in five integer values from standard input (std::cin) using the >> operator. The while loop ensures that only valid inputs are read by testing for


In [10]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Írj egy egyszerű python kódot, ami kíírja a konzolra hogy helló világ
### Response:"""

KATICA_generate(PROMPT)

Generating...
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Írj egy egyszerű python kódot, ami kíírja a konzolra hogy helló világ
### Response:
helló világ = "Hello World!"
print("Helló Világ")
##[End of File]<|endoftext|>
